In [30]:
import pandas as pd

In [31]:
#In the future, we can improve this by being able to read in any/all the datasets

#reads in the data set
data = pd.read_csv(r"Affiliation_Parsing/NeurIPS affiliations/NeurIPSaffiliations-2010-affiliation.csv")
data = pd.read_csv(r"/Users/jhanviganesh/Downloads/xaa.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Unnamed: 0                19 non-null     int64 
 1   Unnamed: 0.1              19 non-null     int64 
 2   first-author              19 non-null     object
 3   first-author-affiliation  19 non-null     object
 4   last-author               19 non-null     object
 5   last-author-affiliation   19 non-null     object
 6   title                     19 non-null     object
dtypes: int64(2), object(5)
memory usage: 1.2+ KB


In [32]:
#drops every column that is not the authors or their affiliations
data = data.drop(data.columns[0:2],axis=1)
data = data.drop(data.columns[4:5],axis=1)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   first-author              19 non-null     object
 1   first-author-affiliation  19 non-null     object
 2   last-author               19 non-null     object
 3   last-author-affiliation   19 non-null     object
dtypes: object(4)
memory usage: 736.0+ bytes


In [33]:
#imports BS4 libraries for webscraping
from re import I
import requests
from bs4 import BeautifulSoup

In [21]:
#Need to add in they/them
#Improve search


def findGender(urlLink):
    gender = ''
    ## put a url where the desciption of the author is avilable
    url = urlLink
    htmlText = requests.get(url).text
    soup = BeautifulSoup(htmlText, 'html.parser')


    # gathers all the p tags from the html
    soupstring = str(soup.find_all("p"))
    shetags = [" she ", " She "," her ", " Her "," hers "," Hers "]
    hetags = [" he ", " He "," his ", " His "," him "," Him "]

    ##checks if male or female pronouns are present in the desciption
    containsfemale = any(femalepronouns in soupstring for femalepronouns in shetags)
    containsmale = any(malepronouns in soupstring for malepronouns in hetags)

    if containsfemale:
        gender = "female"
    elif containsmale:
        gender = "male"
    else:
        gender = "undefined"
    return gender

In [34]:
#returns whether or not a substring exists in a string
def substring_exists(string, sub_string):
    return string.find(sub_string)

In [35]:
#returns a list of the first-authors
def getFirstAuthorList():
    for column in data[['first-author']]:
        authors = data[column]

        '''
        Prints out column name and its values
        '''
        #print('Column Name: '+column)
        #print(authors.values)

    lst_authors = authors.values
    return lst_authors

In [36]:
#imports googlesearch to use requests and BS4 to scrape google
from googlesearch import search

In [25]:
#initializes new column for the gender of the first author
data['first-gender'] = ''

#fills in gender column for first-author column
def genderFirstAuthor():

    #initializes counter that keep track of author index
    counter = 0

    #gets list of the first authors
    lst=getFirstAuthorList()

    #loops through each author in the dataset
    for person in lst:

        '''
        search term for googlesearch
        follows structure of author's name, their affiliation, and .edu
        i.e. Glendon Chin Steven's Institute of Technology .edu
        '''
        query=person+" "+data['first-author-affiliation'].values[counter]+" .edu"

        #gets multiple links based on the search term
        for link in search(query, 5, "en", None):

            #if the link is not .edu, we will not use that link
            if(substring_exists(link, ".edu")!=-1):
                data['first-gender'].values[counter]=findGender(link)
                print("Link: "+link)
                print()
            else:
                print("skipped "+link)
        
        #increments index, moves to the next row
        counter+=1



In [26]:
#As you can see, we need to look into optimizing the findGender function
#I will continue to work on the googlesearch so that it gives the best websites to navigate
print(data[['first-gender']])

   first-gender
0              
1              
2              
3              
4              
5              
6              
7              
8              
9              
10             
11             
12             
13             
14             
15             
16             
17             
18             


In [37]:
def createKeyValue():
    #creates a dictionary for key value pairs
    linkDictionary = {}
    counter = 0
    #temp list for all te edu links for each author
    linkArray = []
    lst=getFirstAuthorList()
    for person in lst:
    
        query=person+" "+data['first-author-affiliation'].values[counter]+" .edu"

        for link in search(query, tld="co.in", num=10, stop=10, pause=2):
            #appends all the edu links of each author in the temp array
            if(substring_exists(link, ".edu")!= -1):
                linkArray.append(link)

        #creates the pair in the dictonary
        linkDictionary[person] = linkArray
        #resets the temp array to be empty
        linkArray =[]
        counter+=1

    return linkDictionary